Working angular sep

In [23]:
#import packages 
import numpy as np
import pandas as pd
import time

In [24]:
ROSAT = pd.read_csv("2rxs_lite.txt", delim_whitespace=True)  
Einstein = pd.read_csv("2sig_J2000_theta.txt", delim_whitespace=True)

RA_ROSAT = ROSAT.iloc[:, 2] #right ascention
DEC_ROSAT = ROSAT.iloc[:, 3] #decllination
SNR_ROSAT = ROSAT.iloc[:, 4] #signal to noise ratio

RA_Einstein = Einstein.iloc[:, 0] #right ascention
DEC_Einstein = Einstein.iloc[:, 1] #decllination
SNR_Einstein = Einstein.iloc[:, 2] #signal to noise ratio

In [46]:
start = time.time() #start a time to see how long it takes

def coor_data(dec1, dec2, ra1, ra2, snr1, snr2): 
    count = 0
    THETA = [] #the angular separation
    DEC1 = []
    DEC2 = []
    RA1 = []
    RA2 = []
    SNR1 = []
    SNR2 = []
    OFFSET = []
    k = 0

    for i in range(np.size(dec1)): 
        tmp1 = np.where( np.absolute(dec1[i] - dec2) < 1) 
        tmp2 = np.where( np.absolute(ra1[i]-ra2) * np.cos(dec2*(np.pi/180))  < 1 ) 
        resultArray = (np.intersect1d(tmp1, tmp2))
        
        if np.size(resultArray) or len(resultArray) == 0: 
            k = k +1
            #continue
        else: 
            for j in resultArray:
                #count+= 1
                theta_rad = np.sin(dec1[i]*np.pi/180) * np.sin(dec2[j]*np.pi/180) + np.cos(dec1[i]*np.pi/180)*np.cos(dec2[j]*np.pi/180)*np.cos(np.absolute(ra1[i]-ra2[j])*np.pi/180) #returns answer in rad
                theta_arcsec = np.arccos(theta_rad) * 206265
                if theta_arcsec < 360: 
                    if snr1[i] >= 2 and snr1[i] < 3:
                        poserr1 = 73 
                    elif snr1[i] >= 3 and snr1[i] < 4:
                        poserr1 = 53
                    elif snr1[i] >= 4 and snr1[i] < 6:
                        poserr1 = 41 
                    elif snr1[i] >= 6:
                        poserr1 = 30
                    else:
                        continue 
                        
                    poserr2 = snr2[j] 
                    confid = theta_arcsec/np.sqrt(poserr1**2 + poserr2**2) #finding theta hat
                    #print(theta_arcsec)
                    #k += 1
                    #print(k)
                    #if confid <= 2.6:
                    THETA.append(theta_arcsec)
                    OFFSET.append(confid)
                    DEC1.append(dec1[i])
                    DEC2.append(dec2[j])
                    RA1.append(ra1[i])
                    RA2.append(ra2[j])
                    SNR1.append(snr1[i])
                    SNR2.append(snr2[j])
                    count = count + 1
    print(count)
    return THETA, OFFSET, DEC1, DEC2, RA1, RA2, SNR1, SNR2, count, k
    
                

In [47]:
#Here is where you fun the function
theta = []
confidence = []
dec_1 = []
dec_2 = []
ra_1 = []
ra_2= []
snr_1 = []
snr_2 = []
fileSize = 0
other = 0


print("Start...")

theta, confidence, dec_1, dec_2, ra_1, ra_2, snr_1, snr_2, fileSize, other = coor_data(DEC_Einstein, DEC_ROSAT, RA_Einstein, RA_ROSAT, SNR_Einstein, SNR_ROSAT)

#theta, dec_E, dec_R, RA_E, RA_R = coor_data(DEC_Einstein, myDEC_ROSAT, RA_Einstein, myRA_ROSAT)
print("End...")
#print(theta)

Start...
0
End...


In [48]:
print("Writing file")

print(fileSize)
print(other)
with open(r'testEandR.txt', 'w') as fp:
        for i in range(fileSize):
            if i == 0:
                fp.write("Theta(arcsec)\tOffset\tDEC-Einstein\tDEC-ROSART\tRA-Einstein\tRA-ROSAT\tSNR-Einstein\tSNR-ROSAT\n")
            else:
                fp.write("%8.1f\t%8.2f\t%f\t%f\t%12.6f\t%12.6f\t%8.2f\t%8.2f\n" %((theta[i]), (confidence[i]), (dec_1[i]), (dec_2[i]), (ra_1[i]), (ra_2[i]), (snr_1[i]), (snr_2[i]) ))
          
print("finished")

#print(f'Numpy: {time.time() - start} seconds')

Writing file
0
46185
finished


Ignore cells below

In [158]:
start = time.time() #start a time to see how long it takes
count = 0

theta =[] #the angular separation
k = 0

for i in range(10): #(np.size(DEC_Einstein)): 
        
        tmp = np.where( ((np.absolute(DEC_Einstein[i] - DEC_ROSAT) < 1)) & ( (np.absolute(RA_Einstein[i]-RA_ROSAT[i]) * np.cos(DEC_ROSAT*(np.pi/180)) )  < 1 ) )  #conditions it need to meet
        #print(type(tmp))
        #print(np.size(tmp))
        if np.size(tmp) ==0: 
                continue
        else: 
                for j in tmp:
                        #print(tmp)
                        count+= 1
                        theta_rad = np.sin(DEC_Einstein[i]*np.pi/180)* np.sin(DEC_ROSAT[j]*np.pi/180) + np.cos(DEC_Einstein[i]*np.pi/180)*np.cos(DEC_ROSAT[j]*np.pi/180)*np.cos(np.absolute(RA_Einstein[i]-RA_ROSAT[j])*np.pi/180) #returns answer in rad
                        #print(theta_rad)
                        theta_arcsec = np.arccos(theta_rad) * 206265
                        #print(theta_arcsec)
                        #print(type(theta_arcsec))
                        if (theta_arcsec < 300).all: 
                                np.append(theta, theta_arcsec)
                                #print(theta)
                                tmp_num = theta_arcsec
                                k += 1

                
print(count)
print(np.size(theta))
#a = np.sin(90*np.pi/180)
#print(a)

ang_sep = np.zeros(count)
#for i in range(count):
    #ang_sep[i] = round(theta[i].value, 3)
#    ang_sep[i] = theta[0][i]

#print(np.shape(tmp))
print("Writing file")

with open(r'test.txt', 'w') as fp:
        for i in range(count):
            if i == 0:
                fp.write("Theta\n")
            else:
                fp.write("%f\n" %(theta[i]))

print(f'Numpy: {time.time() - start} seconds')


0
638
Writing file
Numpy: 0.39554929733276367 seconds


In [51]:
start = time.time() #start a time to see how long it takes

ROSAT = pd.read_csv("2rxs_lite.txt", delim_whitespace=True)  
Einstein = pd.read_csv("2sig_J2000_theta.txt", delim_whitespace=True) #SDSS
#Einstein = pd.read_csv("dr7_sara.csv", sep=',', header = 1) #SDSS

RA_ROSAT = ROSAT.iloc[:, 2] #right ascention
DEC_ROSAT = ROSAT.iloc[:, 3] #decllination
SNR_ROSAT = ROSAT.iloc[:, 4] #signal to noise ratio

RA_Einstein = Einstein.iloc[:, 0] #right ascention
DEC_Einstein = Einstein.iloc[:, 1] #decllination
SNR_Einstein = Einstein.iloc[:, 2] #signal to noise ratio


count = 0

theta = [] #the angular separation
k = 0

for i in range(np.size(DEC_Einstein)): 
        
        #tmp = np.where( ((np.absolute(DEC_Einstein[i] - DEC_ROSAT) < 1)) & ( (np.absolute(RA_Einstein[i]-RA_ROSAT[i]) * np.cos(DEC_ROSAT*(np.pi/180)) )  < 1 ) )  #conditions it need to meet
        #print(type(tmp))
        tmp1 = np.where( np.absolute(DEC_Einstein[i] - DEC_ROSAT) < 1) 
        tmp2 = np.where( np.absolute(RA_Einstein[i]-RA_ROSAT) * np.cos(DEC_ROSAT*(np.pi/180))  < 1 ) 
        resultArray = (np.intersect1d(tmp1, tmp2))
        
        #print(np.size(resultArray))
        if np.size(resultArray) ==0: 
                continue
        else: 
                for j in resultArray:
                #tmp[0]:
                        #j = tmp[0]
                        #print(j)
                        count+= 1
                        theta_rad = np.sin(DEC_Einstein[i]*np.pi/180)* np.sin(DEC_ROSAT[j]*np.pi/180) + np.cos(DEC_Einstein[i]*np.pi/180)*np.cos(DEC_ROSAT[j]*np.pi/180)*np.cos(np.absolute(RA_Einstein[i]-RA_ROSAT[j])*np.pi/180) #returns answer in rad
                        #print(theta_rad)
                        theta_arcsec = np.arccos(theta_rad) * 206265
                        #print(theta_arcsec)
                        #print(type(theta_arcsec))
                        if theta_arcsec < 360: 
                        
                                if SNR_Einstein[i] >= 2 and SNR_Einstein[i] < 3:
                                        poserr1 = 73 
                                elif SNR_Einstein[i] >= 3 and SNR_Einstein[i] < 4:
                                        poserr1 = 53
                                elif SNR_Einstein[i] >= 4 and SNR_Einstein[i] < 6:
                                        poserr1 = 41 
                                elif SNR_Einstein[i] >= 6:
                                        poserr1 = 30
                                else:
                                        continue 
                                        #this is for cases of snr less than 2. Is this not possible or do we ignore them?
                                
                                
                                poserr2 = 1 #SNR_ROSAT[j] 
                                confid = theta_arcsec/np.sqrt(poserr1**2 + poserr2**2) #finding theta hat
                                #print(theta_arcsec)
                                if confid <= 2.6:
                                        theta.append(theta_arcsec)
                                        #d1.append(dec1[i])
                                        #d2.append(dec2[j])
                                        #ra1.append(RA1[i])
                                        #ra2.append(RA2[j])
                                        #snr1.append(SNR_Einstein[i])
                                        #snr2.append(SNR_ROSAT[j])
                                        #print(theta)
                                        tmp_num = theta_arcsec
                                        k += 1

                
print(count)
print(k)
print(np.size(theta))
#a = np.sin(90*np.pi/180)
#print(a)

ang_sep = np.zeros(count)
#for i in range(count):
    #ang_sep[i] = round(theta[i].value, 3)
#    ang_sep[i] = theta[0][i]

#print(np.shape(tmp))
print("Writing file")

with open(r'dr7rxs.txt', 'w') as fp:
        for i in range(k):
            if i == 0:
                fp.write("Theta (arcsec)\n")
            else:
                fp.write("%f\n" %(theta[i]))

print(f'Numpy: {time.time() - start} seconds')

755860
4824
4824
Writing file
Numpy: 140.42655205726624 seconds


In [127]:
i = 50
print(DEC_Einstein[i])
tmp = np.where( ((np.absolute(DEC_Einstein[i] - DEC_ROSAT) < 1)) & ( (np.absolute(RA_Einstein[i]-RA_ROSAT) * np.cos(DEC_Einstein[i]*(np.pi/180)) )  < 1 ) )

57.503447


In [128]:
tmp
DEC_ROSAT[621]

74.7645

In [45]:
print(DEC_ROSAT[527])


70.76032


In [63]:
arr = np.array([16, 12, 14, 15, 16, 17])
arr2 = np.array([12, 15, 16, 10, 26, 12])
# pass condition expression    only
result = np.where((arr[0] > arr2) & (arr[0] > arr2 *1.50))

In [64]:
result

(array([3]),)

In [146]:
i = 50

tmp1 = np.where( np.absolute(DEC_Einstein[i] - DEC_ROSAT) < 1) 
tmp2 = np.where( np.absolute(RA_Einstein[i]-RA_ROSAT) * np.cos(DEC_ROSAT*(np.pi/180))  < 1 ) 
resultArray = (np.intersect1d(tmp1, tmp2))

type(resultArray)

#print(tmp2)

(array([36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68]),)
(array([ 50,  53,  54,  65, 155, 167, 328, 593, 613, 615, 616, 619, 621,
       636]),)
<class 'numpy.ndarray'>


/var/folders/3f/g9qgz71j35z23v4byb9jtltc00031c/T/ipykernel_32108/3182480801.py:5: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  results = np.where( tmp == tmp2)


In [118]:
i = 50
tmp = np.where( (np.absolute(DEC_Einstein[i] - DEC_ROSAT) < 1) )
print(DEC_Einstein[i])
tmp

57.503447


(array([36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
        53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68]),)

In [119]:
#print(DEC_ROSAT[36:68]) 
array([50, 53, 54, 65]),)

In [120]:
i = 50
tmp = np.where( np.absolute(RA_Einstein[i]-RA_ROSAT) * np.cos(DEC_Einstein[i]*(np.pi/180)) < 1 ) 
print("RA of E",RA_Einstein[i])
print("DEC of E",DEC_Einstein[i])
tmp


RA of E 122.823387
DEC of E 57.503447


(array([ 50,  53,  54,  65, 155, 167, 328, 613, 619, 621, 636]),)

In [75]:
RA_ROSAT[50]

122.79224

In [80]:
"""import numpy as np
  
# Create two multidimensional arrays of 
# integer values
np_arr1 = np.array([[6, 13, 22, 7, 12],  
                    [7, 11, 16, 32, 9]])
np_arr2 = np.array([[44, 20, 8, 35, 10],  
                    [98, 23, 42, 6, 13]])
  
# Print the array values
print("\nThe values of the first array :\n", np_arr1)
print("\nThe values of the second array :\n", np_arr2)
  
# Create a new array from two arrays based on
# the conditions
new_arr = np.where(((np_arr1 % 2 == 0) & (np_arr2 % 2 == 1)), 
                   np_arr1, np_arr2)
  
# Print the new array
print("\nThe filtered values of both arrays :\n", new_arr)"""

'import numpy as np\n  \n# Create two multidimensional arrays of \n# integer values\nnp_arr1 = np.array([[6, 13, 22, 7, 12],  \n                    [7, 11, 16, 32, 9]])\nnp_arr2 = np.array([[44, 20, 8, 35, 10],  \n                    [98, 23, 42, 6, 13]])\n  \n# Print the array values\nprint("\nThe values of the first array :\n", np_arr1)\nprint("\nThe values of the second array :\n", np_arr2)\n  \n# Create a new array from two arrays based on\n# the conditions\nnew_arr = np.where(((np_arr1 % 2 == 0) & (np_arr2 % 2 == 1)), \n                   np_arr1, np_arr2)\n  \n# Print the new array\nprint("\nThe filtered values of both arrays :\n", new_arr)'

In [96]:
i = j = 1

theta_rad = np.sin(DEC_Einstein[i]*np.pi/180)* np.sin(DEC_ROSAT[j]*np.pi/180) + np.cos(DEC_Einstein[i]*np.pi/180)*np.cos(DEC_ROSAT[j]*np.pi/180)*np.cos(np.absolute(RA_Einstein[i]-RA_ROSAT[j])*np.pi/180) #returns answer in rad
print(theta_rad)
print(np.arccos(theta_rad)*206265)

0.9999999921701705
25.811704851343713


In [94]:
DEC_Einstein[i]

55.137461

In [95]:
DEC_ROSAT[j]

55.13077

In [130]:
a = [0, 1, 2, 3, 4, 5, 6, 7]
print(np.where((a > 2) & (a < 6) | (a == 7)))

TypeError: '>' not supported between instances of 'list' and 'int'